In [ ]:
!pip install transformers datasets rouge_score

from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from rouge_score import rouge_scorer
import numpy as np

# 1. Load a VERY Small Subset of the Dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")  # Use only 1% of the training data

# 2. Choose a Smaller Model
model_name = 't5-small'  # Can upgrade to 't5-base' for better quality
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 3. Preprocess Data (Optimized for Abstractive Summarization)
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")

    labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length", return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 4. Training (Optimized for Faster Learning)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=3e-5,  # Slightly increased learning rate
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,  # Train for only ONE epoch
    weight_decay=0.01,
    save_steps=10000,
    logging_steps=100,
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

trainer.train()

# 5. Inference (Abstractive Summarization)
def generate_summary(text):
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids,
        max_length=100,
        num_beams=5,  # More beams for better summarization
        early_stopping=True,
        length_penalty=1.2,  # Encourages slightly shorter summaries
        temperature=0.9  # Encourages more diverse output
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 6. Examples
texts = [
    "Climate change is increasingly affecting global agriculture, posing significant challenges to food security and livelihoods. Rising temperatures, altered precipitation patterns, and increased frequency of extreme weather events are some of the key factors impacting crop yields and livestock productivity. For instance, prolonged droughts can lead to water scarcity, affecting irrigation and reducing crop growth. Similarly, excessive rainfall can cause flooding, damaging crops and soil health.",

    "The Sri Lankan government said on Saturday that the recent killings in the country, including that of a prominent underworld figure, appear to be an attempt to undermine the ongoing investigations into high-profile criminal cases such as the 2019 Easter Sunday suicide bombings. The government’s response came following this week’s courtroom shooting that left the underworld figure dead.  The incident sent shockwaves across the country, leading to the opposition probing the government’s measures to tackle the spate of violence."
]

for i, text in enumerate(texts):
    print(f"\n**Original Text {i+1}:**\n{text}\n")
    print(f"**Generated Summary {i+1}:**\n{generate_summary(text)}\n")


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6d9ac8e737979855e005929425cf1491ee10ebdb6429e82dbae71602bbd4d6d5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

Step,Training Loss
100,3.610000
200,1.095200


Step,Training Loss
100,3.610000
200,1.095200
300,1.029800



**Original Text 1:**
Climate change is increasingly affecting global agriculture, posing significant challenges to food security and livelihoods. Rising temperatures, altered precipitation patterns, and increased frequency of extreme weather events are some of the key factors impacting crop yields and livestock productivity. For instance, prolonged droughts can lead to water scarcity, affecting irrigation and reducing crop growth. Similarly, excessive rainfall can cause flooding, damaging crops and soil health.



/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


**Generated Summary 1:**
Climate change is increasingly affecting global agriculture, posing significant challenges to food security and livestock productivity. Rising temperatures, altered precipitation patterns, and increased frequency of extreme weather events are key factors affecting crop yields and livestock productivity. For example, prolonged droughts can lead to flooding, flooding and reducing crop growth. excessive rainfall can cause flooding, reducing crop growth.


**Original Text 2:**
The Sri Lankan government said on Saturday that the recent killings in the country, including that of a prominent underworld figure, appear to be an attempt to undermine the ongoing investigations into high-profile criminal cases such as the 2019 Easter Sunday suicide bombings. The government’s response came following this week’s courtroom shooting that left the underworld figure dead.  The incident sent shockwaves across the country, leading to the opposition probing the government’s measure